In [ ]:
!pip install jupyter_dash
!pip install dash-bootstrap-components
!pip install jupyter-dash

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash import html


import plotly.graph_objects as go

import dash_html_components as html
from dash.dependencies import Input, Output, State

from IPython.display import display, HTML

!pip install torch torchvision torchaudio


# Setup RoBERTa

In [ ]:
!pip install transformers

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
example = "I think this is going to fail"

# Run for Roberta Model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'NEGATIVE' : scores[0],
    'NEUTRAL' : scores[1],
    'POSITIVE' : scores[2]}
print(scores_dict)

 works really well!

In [ ]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'NEGATIVE' : scores[0],
        'NEUTRAL' : scores[1],
        'POSITIVE' : scores[2]
    }
    return scores_dict

In [ ]:
scores_dict["NEGATIVE"]

In [ ]:
def find_sentiment_scores(sentence):
    sentiment = polarity_scores_roberta(sentence)
    return scores_dict

user_input = input("Enter a sentence: ")
result = find_sentiment_scores(user_input)
print(result)


In [ ]:
def analyze_sentiment(sentence):
    text = find_sentiment_scores(sentence)

    positive_percentage = text["POSITIVE"] * 100
    neutral_percentage = text["NEUTRAL"] * 100
    negative_percentage = text["NEGATIVE"] * 100

    sentiment_distribution = {
        "Positive": positive_percentage,
        "Negative": negative_percentage,
        "Neutral": neutral_percentage
    }

    return sentiment_distribution


analyze_sentiment("Hello World")

# The dash code

In [ ]:

# Define the navbar component
navbar = dbc.Navbar(
    [
        dbc.Col(html.Div(dbc.NavLink("Home", href="#", style={"color": "red"})), width={"auto": "10rem"}),
        dbc.Col(html.Div(dbc.NavLink("More", href="#", style={"color": "green"})), width={"auto": "10rem"}),
        dbc.Col(html.Div(dbc.NavLink("About Us", href="#", style={"color": "blue"})), width={"auto": "10rem"}),
    ],
    color="darkblue",
    className="navbar-custom",
    dark=True,
    style={"justify-content": "end"},
)



# Create the Dash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Set up the app layout
app.layout = html.Div(
    children=[
        navbar,  # include the navbar here
        html.H1("Sentiment Analysis"),
        dcc.Textarea(id="input-text", placeholder="Enter a sentence"),
        html.Button("Analyze", id="analyze-button", n_clicks=0),
        html.Div(id="page-content"),  # a container for the content to be updated based on user input
        dcc.Graph(id="positive-pie-chart"),
        dcc.Graph(id="positive-bar-chart"),
        dcc.Graph(id="negative-pie-chart"),
        dcc.Graph(id="negative-bar-chart"),
        dcc.Graph(id="neutral-pie-chart"),
        dcc.Graph(id="neutral-bar-chart"),
    ],
)


@app.callback(
    [
        Output("page-content", "children"), #this will be updated
        Output("positive-pie-chart", "figure"),
        Output("positive-bar-chart", "figure"),
        Output("negative-pie-chart", "figure"),
        Output("negative-bar-chart", "figure"),
        Output("neutral-pie-chart", "figure"),
        Output("neutral-bar-chart", "figure"),
    ],
    [
        Input("analyze-button", "n_clicks"), #will trigger the callback function when its value changes (i.e., when the button is clicked)
    ],
    State("input-text", "value"),
)
def update_page_content(n_clicks, input_text):
    if not input_text:
        return "", {}, {}, {}, {}, {}, {}

    example = input_text

    # Run for Roberta Model
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'NEGATIVE' : scores[0],
        'NEUTRAL' : scores[1],
        'POSITIVE' : scores[2]}

    # get probability scores of the input text using RoBERTa
    def polarity_scores_roberta(example):
        encoded_text = tokenizer(example, return_tensors='pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'NEGATIVE' : scores[0],
            'NEUTRAL' : scores[1],
            'POSITIVE' : scores[2]
        }
        return scores_dict

    # setup function for the analyzing
    def find_sentiment_scores(sentence):
        sentiment = polarity_scores_roberta(sentence)
        return scores_dict


    # finally analyze the sentiment
    def analyze_sentiment(sentence):
        text = find_sentiment_scores(sentence)

        positive_percentage = text["POSITIVE"] * 100
        neutral_percentage = text["NEUTRAL"] * 100
        negative_percentage = text["NEGATIVE"] * 100

        sentiment_distribution = {
            "Positive": positive_percentage,
            "Negative": negative_percentage,
            "Neutral": neutral_percentage
        }

        return sentiment_distribution


    sentiment_distribution = analyze_sentiment(input_text)

    main_sentiment = max(sentiment_distribution, key=sentiment_distribution.get)
    main_sentiment_percentage = sentiment_distribution[main_sentiment]

    page_content = html.Div(
        children=[
            html.H1(f"{main_sentiment} Sentiment"),
            html.H3("Sentiment Distribution:"),
            html.Ul(
                children=[
                    html.Li(f"{sentiment}: {percentage:.2f}%")
                    for sentiment, percentage in sentiment_distribution.items()
                ]
            )
        ]
    )

    # Pie chart for sentiment distribution
    pie_chart_data = [
        {"Sentiment": sentiment, "Percentage": percentage}
        for sentiment, percentage in sentiment_distribution.items()
    ]
    pie_chart = px.pie(pie_chart_data, values="Percentage", names="Sentiment", title="Sentiment Distribution")

    # Bar chart for sentiment distribution
    bar_chart_data = {"Sentiment": list(sentiment_distribution.keys()), "Percentage": list(sentiment_distribution.values())}
    bar_chart = px.bar(bar_chart_data, x="Sentiment", y="Percentage", title="Sentiment Distribution")

    return (
        page_content,
        pie_chart,
        bar_chart,
        {},
        {},
        {},
        {},
    )

if __name__ == '__main__':
    app.run_server(mode='external')


Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>

# Actual code we're working with

In [ ]:
# Define the navbar component
navbar = dbc.Navbar(
    [
        dbc.Nav(
                        [
                dbc.NavLink("T E X T M O O D", href="#", style={"color": "#599FC7"}),
                dbc.NavLink("Online Sentiment Analyzer", href="#", style={"color": "#599FC7"}),
                dbc.NavLink("About Us", href="#", style={"color": "#599FC7"}),
            ],
            className="ml-auto",
            navbar=True,
        )

    ],
    color = "black",
    className="navbar-custom",
    dark=True,
    style={"backgroundColor": "#585252"},
    )

navbar.container_style = {"justify-content": "space-between"}  # Add this line to align entire navbar contents

navbar.children[0].style = {"display": "flex", "align-items": "center"}  # Add this line to align NavLinks vertically

navbar.children[0].children[0].style = {"margin-right": "1rem"}  # Add margin between NavLinks
navbar.children[0].children[1].style = {"margin-right": "1rem"}
navbar.children[0].children[2].style = {"margin-right": "1rem"}


# Create the Dash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


# Set up the app layout
app.layout = html.Div(
    style={
        'textAlign': 'center',
        'border': '1px solid black',
        'backgroundColor': '#4E6894'  # Set the background color to light gray

    },
    children=[
        navbar,  # include the navbar here
        html.H1("T E X T M O O D"),
        html.H3("Find out the sentiment behind any text!"),
        dcc.Textarea(id="input-text", placeholder="Enter a sentence"),
        html.Button("Analyze", id="analyze-button", n_clicks=0),
        html.Div(id="page-content"),
        html.Div(
            className="bar-chart-container",
            children=[
                dcc.Graph(id="positive-pie-chart"),
                dcc.Graph(id="positive-bar-chart"),
            ],
            style={'width': '60%', 'margin': '0 auto'}  # Set width and center the container

        )
    ],
)

@app.callback(
    [
        Output("page-content", "children"), #this will be updated
        Output("positive-pie-chart", "figure"),
        Output("positive-bar-chart", "figure"),
    ],
    [
        Input("analyze-button", "n_clicks"), #will trigger the callback function when its value changes (i.e., when the button is clicked)
    ],
    State("input-text", "value"),
)
def update_page_content(n_clicks, input_text):
    if input_text is None or input_text.strip() == "":
        return ""

    example = input_text

    # Run for Roberta Model
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'NEGATIVE' : scores[0],
        'NEUTRAL' : scores[1],
        'POSITIVE' : scores[2]}

    # get probability scores of the input text using RoBERTa
    def polarity_scores_roberta(example):
        encoded_text = tokenizer(example, return_tensors='pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'NEGATIVE' : scores[0],
            'NEUTRAL' : scores[1],
            'POSITIVE' : scores[2]
        }
        return scores_dict

    # setup function for the analyzing
    def find_sentiment_scores(sentence):
        sentiment = polarity_scores_roberta(sentence)
        return scores_dict


    # finally analyze the sentiment
    def analyze_sentiment(sentence):
        text = find_sentiment_scores(sentence)

        positive_percentage = text["POSITIVE"] * 100
        neutral_percentage = text["NEUTRAL"] * 100
        negative_percentage = text["NEGATIVE"] * 100

        sentiment_distribution = {
            "Positive": positive_percentage,
            "Negative": negative_percentage,
            "Neutral": neutral_percentage
        }

        return sentiment_distribution


    sentiment_distribution = analyze_sentiment(input_text)

    main_sentiment = max(sentiment_distribution, key=sentiment_distribution.get)
    main_sentiment_percentage = sentiment_distribution[main_sentiment]

    page_content = html.Div(
        children=[
            html.H1(f"{main_sentiment} Sentiment"),
            html.H3("Sentiment Distribution:"),
            html.Ul(
                children=[
                    html.Li(f"{sentiment}: {percentage:.2f}%")
                    for sentiment, percentage in sentiment_distribution.items()
                ]
            )
        ]
    )


    # Pie chart for sentiment distribution
    pie_chart_data = [
        {"Sentiment": sentiment, "Percentage": percentage, "Color": color}
        for sentiment, percentage, color in zip(sentiment_distribution.keys(), sentiment_distribution.values(), ["35DD35", "B21C1C", "31AFD9"])
    ]

    pie_chart = px.pie(pie_chart_data, values="Percentage", names="Sentiment", title="Distribution of positve, neutral and negative sentiment:")
    pie_chart.update_traces(marker=dict(colors=[data["Color"] for data in pie_chart_data]))


    # Bar chart for sentiment distribution
    bar_chart_data = {"Sentiment": list(sentiment_distribution.keys()),
                      "Percentage": list(sentiment_distribution.values()),
                      "Color": ["#35DD35", "#B21C1C", "#31AFD9"]}

    #bar_chart = px.bar(bar_chart_data, x="Sentiment", y="Percentage", title="Distribution of positve, neutral and negative sentiment:")
    bar_chart = go.Figure(data=[
                go.Bar(
                    x=bar_chart_data["Sentiment"],
                    y=bar_chart_data["Percentage"],
            marker=dict(color=bar_chart_data["Color"]),
        )
    ])
    bar_chart.update_layout(title="Distribution of positive, neutral, and negative sentiment")




    return (
        page_content,
        pie_chart,
        bar_chart,
    )


if __name__ == '__main__':
    app.run_server(mode='external')


Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>

# Not needed anymore

In [ ]:
# Define the navbar component
navbar = dbc.Navbar(
    [
        dbc.Nav(
                        [
                dbc.NavLink("T E X T M O O D", href="#", style={"color": "#B21C1C"}),
                dbc.NavLink("Online Sentiment Analyzer", href="#", style={"color": "#B21C1C"}),
                dbc.NavLink("About Us", href="#", style={"color": "#B21C1C"}),
            ],
            className="ml-auto",
            # navbar=True,
        )

    ],
    color = "black",
    className="navbar-custom",
    dark=True,
    style={"backgroundColor": "#D9D9D9"},
    )

navbar.container_style = {"justify-content": "space-between"}  # Add this line to align the entire navbar contents

navbar.children[0].style = {"display": "flex", "align-items": "center"}  # Add this line to align the NavLinks vertically

navbar.children[0].children[0].style = {"margin-right": "1rem"}  # Add margin between NavLinks
navbar.children[0].children[1].style = {"margin-right": "1rem"}
navbar.children[0].children[2].style = {"margin-right": "1rem"}


In [ ]:
def analyze_sentiment(sentence):
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(sentence)

    positive_percentage = sentiment_scores["pos"] * 100
    negative_percentage = sentiment_scores["neg"] * 100
    neutral_percentage = sentiment_scores["neu"] * 100

    sentiment_distribution = {
        "Positive": positive_percentage,
        "Negative": negative_percentage,
        "Neutral": neutral_percentage

In [ ]:
    # Pie chart for sentiment distribution
    pie_chart_data = [
        {"Sentiment": sentiment, "Percentage": percentage}
        for sentiment, percentage in sentiment_distribution.items()
    ]
    pie_chart = px.pie(pie_chart_data, values="Percentage", names="Sentiment", title="Sentiment Distribution")